In [1]:
import os
import pickle
import cv2
import matplotlib
import tensorflow as tf
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
import datetime as dt

In [2]:
def loadPreprocess():
    ### Load In Data
    (trainX, trainY), (testX, testY) = tf.keras.datasets.cifar10.load_data()
    trainY = tf.keras.utils.to_categorical(trainY)
    testY = tf.keras.utils.to_categorical(testY)
    return [trainX/255, trainY, testX/255, testY]

In [3]:
trainX, trainY, testX, testY = loadPreprocess()

In [4]:
from tensorflow.keras.models import model_from_json
#load json and create model

json_file = open('Models/model_fine_labels.json', 'r')

loaded_model_json = json_file.read()

json_file.close()

model = model_from_json(loaded_model_json)
#load weights into new model

model.load_weights("Models/model_fine_labels.h5") 
print("Loaded model from disk")


Loaded model from disk


In [5]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_4 (Batch (None, 32, 32, 3)         12        
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
gaussian_noise_9 (GaussianNo (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 28, 28, 32)        9248      
_________________________________________________________________
dropout_15 (Dropout)         (None, 28, 28, 32)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 26, 26, 64)        18496     
_________________________________________________________________
gaussian_noise_10 (GaussianN (None, 26, 26, 64)       

In [6]:
testY

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)

In [7]:
model.compile(optimizer = tf.keras.optimizers.Adam(0.001),
              loss = "categorical_crossentropy",  metrics= ['accuracy'])
model.evaluate(testX, testY, verbose = 1)

10000/10000 [==============================] - 5s 472us/sample - loss: 0.5875 - accuracy: 0.8282


[0.5875392208099365, 0.8282]

In [12]:
extractor  = tf.keras.Model(inputs = model.input,
                         outputs = model.get_layer(model.layers[-3].name).output
                        )
model.compile(optimizer = tf.keras.optimizers.Adam(), 
               loss = "categorical_crossentropy",  metrics= ['accuracy']
              )

In [16]:
train_featX = extractor.predict(trainX)
test_featX = extractor.predict(testX)

In [25]:
### Decsion Tree Classifier
from sklearn.tree import DecisionTreeClassifier
dtr_clf = DecisionTreeClassifier()
dtr_clf.fit(train_featX, trainY)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [23]:

from sklearn.svm import SVC
svc_clf = SVC()
svc_clf.fit(train_featX, np.argmax(trainY, axis =1 ))

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [26]:
from sklearn.metrics import accuracy_score

pred1 = dtr_clf.predict(test_featX)
dtr_acc = accuracy_score(testY, pred1)

pred2 = svc_clf.predict(test_featX)
svc_acc = accuracy_score(testY, pred1)
print("Decision Tree Accuracy: ", dtr_acc)
print("Support Vector Machine Accuracy: ", svc_acc)

Decision Tree Accuracy:  0.795
Support Vector Machine Accuracy:  0.795


In [28]:
from joblib import dump, load 
###Saving shallow learning models
dump(dtr_clf, "shallow_models\dtcClf.joblib")
dump(svc_clf, "shallow_models\svcClf.joblib")
print("Models saved!")

Models saved!
